<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221114_ML_RandomForest_%EC%A4%91%EA%B3%A0%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 중고자 가격 예측

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho
file_url = 'https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/car.csv'
df = pd.read_csv(file_url) # 데이터셋 읽기

|name|year|selling_price|km_driven|fuel|seller_type|transmission|owner|mileage|engine|max_power|torque|seats|
|-|-|-|-|-|-|-|-|-|-|-|-|-|
|이름|생산년|판매가|주행거리|연료|판매자 유형|변속기|차주 변경 내역|마일리지(연비)|배기량|최대출력|토크|인승|

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()   # 결측치, 데이터 타입

In [ ]:
#@title 결측치 
missing_value = df.isnull().sum()
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
ax = missing_value[missing_value.gt(0)].plot(xlim=(0, 250), kind='barh')
for p in ax.patches:
    ax.annotate(str(p.get_width()), (p.get_width() * 1.005, p.get_y() + 0.15))

In [ ]:
missing_value = df.isnull().mean()
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
ax = missing_value[missing_value.gt(0)].plot(xlim=(0, 0.5 ), kind='barh')
for p in ax.patches:
    ax.annotate(str(p.get_width()), (p.get_width() * 1.005, p.get_y() + 0.15))

In [ ]:
#@title 단위 처리
# 단위 때문에 범주형(object 분류되는 데이터..)
df.engine.unique()

In [ ]:
pd.options.display.float_format = "{:,.2f}".format
df.describe()

In [ ]:
df.boxplot()

In [ ]:
# subplots
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1 = df.selling_price.plot.box(ax=ax1)
ax2 = df.km_driven.plot.box(ax=ax2)
plt.show()

## 전처리

### 단위 변환

In [ ]:
df.describe(include=['O'])

In [ ]:
# mileage, engine, max_-power -> (숫자) (단위) 

In [ ]:
# df.(칼럼).str : (행단위로) 문자열을 처리하는 메소드(함수)들을 불러올 수 있음
# df.engine.str.split() # 리스트로 쪼개짐
df.engine.str.split(expand=True).head()     # 열로 쪼개짐

In [ ]:
df[['engine', 'engine_unit']] = df.engine.str.split(expand=True)
df.head()

In [ ]:
df.engine.head()

In [ ]:
df.engine.astype('float32')

In [ ]:
df.engine = df.engine.astype('float32')
df.engine.head()

In [ ]:
df.engine_unit.unique()

In [ ]:
df.drop('engine_unit', axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.max_power.head()

In [ ]:
df[['max_power', 'max_power_unit']] = df.max_power.str.split(expand=True)
df.max_power = df.max_power.astype('float32')

In [ ]:
df.max_power[df.max_power == 'bhp']
df.max_power.iloc[0]

In [ ]:
def handle_float(value):
    try:
        # num = float(value)
        # return num
        return float(value)
    except ValueError:
        return np.NaN

In [ ]:
df.max_power = df.max_power.apply(handle_float)

In [ ]:
df.max_power.head()

In [ ]:
df2 = pd.read_csv(file_url)
df2.max_power.head()

In [ ]:
# https://regexr.com/
df2.max_power.str.extract('([\d\.]+)').astype('float').head()

In [ ]:
df.max_power_unit.unique()

In [ ]:
df.drop('max_power_unit', axis = 1, inplace = True) 

In [ ]:
df.columns

In [ ]:
df.mileage.unique() # kmpl (km/l), km/kg -> 연비 

In [ ]:
df[['mileage', 'mileage_unit']] = df.mileage.str.split(expand = True) 
df.columns

In [ ]:
df[['mileage', 'mileage_unit']].head()

In [ ]:
df.mileage = df.mileage.astype('float32') 
df.mileage.head() 

In [ ]:
df.mileage_unit.unique() 

In [ ]:
df.fuel.unique() # Patrol, Diesel -> L / LPG, CNG -> KG 

In [ ]:
fuels = {'Petrol' : 80.43,
         'Diesel' : 73.56,
         'LPG' : 40.85,
         'CNG' : 44.23} 

# apply(function, axis = 1) -> 행의 데이터를 다 쓸 수 있다
def handle_mileage(x): 
    return x.mileage / fuels[x.fuel]          

In [ ]:
df.mileage = df.apply(handle_mileage, axis = 1) 
df.mileage.head() 

In [ ]:
df.drop('mileage_unit', axis = 1, inplace = True) 
df.columns

In [ ]:
df.torque.head() 

In [ ]:
df.torque.unique() 

In [ ]:
df.torque = df.torque.str.upper() 

In [ ]:
# 단위를 뽑아내기 위해 사용하는 함수(torque 열) 
def handle_torque_unit(x):
    if 'NM' in str(x):  # '문자열A' in '문자열 B' -> 문자열 B에 A를 포함하고 있나? - T/F
        return 'Nm'     # Nm -> 단위
    # elif 'KGM' in str(x): 웬만하면 elif 쓰지 마세요
    if 'KGM' in str(x):  # x -> KGM? -> kgm 반환 값
        return 'kgm'
    # ---------
    # ? -> None (return None)

In [ ]:
df['torque_unit'] = df.torque.apply(handle_torque_unit) 
df.torque_unit.unique() 

In [ ]:
df[df.torque_unit.isna()].torque.unique() # Nm 100보다 큰 값, kgm 100미만

In [ ]:
df.torque_unit.fillna('Nm', inplace=True)

In [ ]:
df.torque_unit.unique()

In [ ]:
# ([0-9\.]+) find(find_one) / find_all
# str.extract = 맨처음 검색되는 첫번째만. 2개 이상일 경우에는 1번째만. 없으면? None. -> nan.
# str.extract_all = 리스트(여러개) []
df.torque.str.extract('([\d\.]+)').astype('float64')

In [ ]:
df.torque = df.torque.str.extract('([\d\.]+)').astype('float64')
df.torque.head()

In [ ]:
df.torque_unit.unique()
# 9.8066 kgm -> nm

In [ ]:
# 단위 변환
def handle_torque_trans(x):
    return x.torque * 9.8066 if x.torque_unit == 'kgm' else x.torque

df.torque = df.apply(handle_torque_trans, axis=1)

In [ ]:
df.torque.head()

In [ ]:
df.drop('torque_unit', axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.name

In [ ]:
# 맨 첫 단어만.
df.name = df.name.str.split(expand=True)[0]
df.name.unique()

In [ ]:
df.name = df.name.replace('Land', 'Land Rover') 
df.name.unique() 

In [ ]:
# 결측치의 평균
df.isna().mean()
# df.isna().mean().plot.barh()

In [ ]:
df.dropna(inplace=True)
len(df) # 평균치 치환도 가능

In [ ]:
df.isna().mean().plot.barh()

## 범주형 변수 변환

In [ ]:
df.describe(include = ['O'])

In [ ]:
df = pd.get_dummies(df,
                    columns=['name', 'fuel', 'seller_type', 'transmission', 'owner'],
                    drop_first=True)
df

In [ ]:
df.info()

### 훈련셋 & 시험셋 

In [ ]:
from sklearn.model_selection import train_test_split

# 중고차의 판매가격 -> selling_price (y)
X = df.drop('selling_price', axis=1)
y = df.selling_price
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=100)

In [ ]:
# 연속형 변수 ; RandomForestRegressor
# 범주형 변수 : RandomForestClassifier

from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(random_state = 100)

In [ ]:
model.fit(X_train, y_train) # 학습
train_pred = model.predict(X_train) # 훈련셋 예측 (잘 학습되었나? 과최적화, 오버피팅?)
test_pred = model.predict(X_test) # 시험셋 예측 (잘 예측하나?, 언더피팅) 

In [ ]:
# 분류하는 문제 : accurancy
# 수치 예측 -> RMSE
from sklearn.metrics import mean_squared_error
print(
    # 실제값, 예측값 -> 에러를 비교
    "train_rmse : ", mean_squared_error(y_train, train_pred, squared=False),
    "test_rmse : ", mean_squared_error(y_test, test_pred) ** 0.5 #1/2
)

## k-Fold 교차검증 

* 교차검증: 'train_test_split' 훈련셋 & 시험셋 ->  random.state ??? 1버전의 훈련셋?<br> 다양한 훈련셋 / 시험셋을 통해서 모델의 신뢰성을 높이기 위한 평가 과정

* K-폴드 교차검증: 데이터를 K개를 쪼개서 그 중에 하나를 시험셋으로 선택하는 과정을 반복 -> 평균치 

| A | B | C | D | E |
|-|-|-|-|-| 
|훈|훈|훈|훈|훈|
|훈|훈|훈|시|훈|
|훈|훈|시|훈|훈|
|훈|시|훈|훈|훈|
|시|훈|훈|훈|훈| 

=> accurance_score? rmse? =? 평균


In [ ]:
from sklearn.model_selection import KFold

In [ ]:
df.index

In [ ]:
# df.reset_index() 이렇게만 하면 컬럼하나 생김 
# df.reset_index(drop = True) 
df.reset_index(drop = True, inplace = True) 

In [ ]:
df.index

In [ ]:
# 한계 : test_train_split? -> 랜덤인데? 1번만 한다 -> 다른 조합이면 값이 다를까?
# 여러 조합으로 해봐야 한다 
# 훈,훈,훈,훈,시 / 훈,훈,훈,시,훈 / (조합을 다양하게 해봐서 해당 지표들의 평균?) 
# K폴드 교차검증 (여러개의 시험셋 / 훈련셋을 비교해보자 -> 검증) 
kf = KFold(n_splits = 5) # KFFold 객체 

In [ ]:
X = df.drop('selling_price', axis=1)
y = df['selling_price']

In [ ]:
for i, j in kf.split(X):
    print(f"i : {i}") # 훈련셋의 인덱스
    print(f"j : {j}") # 시험셋의 인덱스

In [ ]:
train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X): # 반복 <- 5기준으로 나뉜 K-Fold
    X_train, X_test = X.loc[train_index], X.loc[test_index] # 독립변수들의 훈련셋/시험셋
    y_train, y_test = y[train_index], y[test_index] # 종속변수의 훈련셋/시험셋

    model = RandomForestRegressor(random_state=100)
    model.fit(X_train, y_train) # 학습
    train_pred = model.predict(X_train) # 훈련셋 예측 (잘 학습되었나? 과최적화, 오버피팅?)
    test_pred = model.predict(X_test) # 시험셋 예측 (잘 예측하나?, 언더피팅)

    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred) ** 0.5 # ** 1/2
    
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)

In [ ]:
train_rmse_total, test_rmse_total

In [ ]:
print(
    # 실제값, 예측값 -> 에러를 비교
    "train_rmse : ", sum(train_rmse_total) / len(train_rmse_total),
    "test_rmse : ", sum(test_rmse_total) / len(test_rmse_total)
)

## 하이퍼 파라미터 튜닝

* n_estimators (기본값 100) : 랜덤포레스트를 구성하는 결정트리 개수 
  - 너무 많거나 적으면? : 성능과 예측력에 영향 

* max_depth (결정트리 유사): 각 트리의 최대 깊이(단계) 
  - 오버피팅 / 언더피팅 : max_depth 
  - 커지면? -> 오버피팅, 작으면 -> 언더피팅 (학습이 잘 안됐다) 

* min_samples_split : 최소 몇 개의 데이터가 노드에 속하게 할 것인지 

* min_samples_leaf : (최종적으로 나눠질) 최소 몇 개의 데이터가 노드에 속하게 할 것인지 

* n_jobs : 병렬 처리 시 사용할 CPU 코어 수

In [ ]:
train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X): # 반복 <- 5기준으로 나뉜 K-Fold
    X_train, X_test = X.loc[train_index], X.loc[test_index] # 독립변수들의 훈련셋/시험셋
    y_train, y_test = y[train_index], y[test_index] # 종속변수의 훈련셋/시험셋
    model = RandomForestRegressor(
        n_estimators = 300,
        max_depth = 50,
        min_samples_split = 5,
        min_samples_leaf = 1,
        n_jobs= -1,
        random_state = 100
    )
    model.fit(X_train, y_train) # 학습
    train_pred = model.predict(X_train) # 훈련셋 예측 (잘 학습되었나? 과최적화, 오버피팅?)
    test_pred = model.predict(X_test) # 시험셋 예측 (잘 예측하나?, 언더피팅)
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred) ** 0.5 # ** 1/2
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)

In [ ]:
print(
    # 실제값, 예측값 -> 에러를 비교
    "train_rmse : ", sum(train_rmse_total) / len(train_rmse_total),
    "test_rmse : ", sum(test_rmse_total) / len(test_rmse_total)
)